In [1]:
import numpy as np
import math
import pandas as pd
from itertools import compress
import matplotlib.pyplot as plt

In [2]:
def reduced_adj(adj,i):
    """
    Function which deletes the ith row and column in an adjacency matrix adj
    """
    adj2 = adj.copy()
    adj2 = np.delete(adj2,i-1,0)
    adj2 = np.delete(adj2,i-1,1)
    return adj2
            
def relabel(adj): 
    """
    Function which implementes the graph relabelling algorithm in Snyder and Shen (2019)
    Relabels node numbers to pass into the Dynamic Programming algorithm.
    """                  
    adj2 = adj.copy()
    U = list(range(1,adj.shape[0]+1))
    L =[]
    label = [0] * adj.shape[0]
    for k in range(1,adj.shape[0]+1):
        for i in range(adj2.shape[0]):
            if sum(adj2[i,:])<=1 :
                adj2 = reduced_adj(adj2,i+1)
                label[U[i]-1] = k
                L.append(U[i])
                del U[i]
                break
    adj2 = adj.copy()
    adj3 = np.zeros(adj2.shape)
    U = list(range(adj2.shape[0]+1))
    for i in range(adj2.shape[0]):
        for j in range(adj2.shape[0]):
            if adj2[i,j]==1:
                adj3[label[i]-1,label[j]-1]=1
                
    return adj3,label

def create_adj(x,y,dim):
    """
    creates a graph adjacency matrix of the supply chain from link vectors x and y 
    """
    adj = np.zeros((dim,dim))
    for k in range(len(x)):
        adj[x[k]-1,y[k]-1] = 1
        adj[y[k]-1,x[k]-1] = 1
    return adj

def print_problem(old_labels,holding_cost,processing_time,echelon,new_labels):
    """
    Prints a summary of the inputs to the optimization problem
    """
    print("Old Labels/Stage:   ",old_labels)
    print("Echelon(Supplier=1):",echelon)
    print("Local Holding Cost: ",holding_cost)
    print("Processing Time:    ",processing_time)
    print("New Labels/Stage:   ",new_labels)

def reorder_labels(holding_cost,processing_time,echelon,by):
    """
    Function to re-order the stage holding costs, processing times and echelon values in ascending 
    order of their corresponding 'by' argument elements
    """
    hc_new = [holding_cost for _,holding_cost in sorted(zip(k,holding_cost))]
    pt_new = [processing_time for _,processing_time in sorted(zip(k,processing_time))]
    ech_new = [echelon for _,echelon in sorted(zip(k,echelon))]
    k_new = sorted(by)
    return hc_new,pt_new,ech_new,k_new,by

def higher_linked(i,adj):
    """
    boolean function which checks if there is a linked node(stage) in the graph with a higher 
    index number or not
    """             
    row = adj[i-1,:]              
    for j in range(len(row)):    
        if row[j]==1:           
            if j>i-1:
                p = j+1           
                return True,p
    return False

def lower_linked(i,adj):
    """
    boolean function which checks if there is a linked node(stage) in the graph with a lower 
    index number or not
    """             
    row = adj[i-1,:]
    p = []
    for j in range(1,len(row)+1):    
        if row[j-1]==1:           
            if j<i:
                p.append(j)
    if len(p) > 0:
        return True,p
    else:
        return False

def downstream(i,j): 
    """
    function to check if stage i is downstream of stage j in the supply chain graph
    A stage i is downstream if the echelon value of stage i is greater than that of another stage j
    """
    if echelon[i-1] > echelon[j-1]:
        return True
    return False

def upstream(i,j):
    """
    function to check if stage i is downstream of stage j in the supply chain graph
    A stage i is downstream if the echelon value of stage i is lesser than that of another stage j
    """
    if echelon[i-1] < echelon[j-1]:
        return True
    return False

def supply_node(n):
    """
    Checks if node n is a supply node in the graph. Supply Chain graphs can 
    have supply, demand or intermediary stages.
    """
    if n in supply_stage:
        return True
    return False

def demand_node(n):
    """
    Checks if node n is a demand node in the graph. Supply Chain graphs can 
    have supply, demand or intermediary stages.
    """
    if n in demand_stage:
        return True
    return False

def cost(k,S,SI,adj,holding_cost,processing_time,sigma,Z):
    """
    Function implementing the cost function of the Dynamic Programming tree algorithm
    """
    term1 = holding_cost[k-1]*Z[k-1]*sigma[k-1]*math.sqrt(SI+processing_time[k-1]-S)
    if not lower_linked(k,adj):
        return term1
    elif lower_linked(k,adj)[0]:
        links = lower_linked(k,adj)[1]
        result = term1
        for j in links:
            if downstream(j,k):
                inter = []
                for l in range(S,M[j-1] - processing_time[j-1]+1):
                    inter.append(theta(j,"i",l,l)[0])
                if len(inter)!=0:
                    inter_min = min(inter)
                else:
                    inter_min = 0
                result = result + inter_min
            if upstream(j,k):
                inter=[]
                for l in range(0,SI+1):
                    inter.append(theta(j,"o",l,l)[0])
                if len(inter)!=0:
                    inter_min = min(inter)
                else:
                    inter_min = 0
                result = result + inter_min
        return result
    
def theta(k,direction,S,SI):
    """
    Function implementing the theta function of the Dynamic Programming tree algorithm
    """
    if direction == "o":
        if supply_node(k):
            range_SI = list(compress(supply_stage_SI, list(map(lambda x:x==k,supply_stage))))
        else:
            range_SI = list(range(max(0,S-processing_time[k-1]),M[k-1]-processing_time[k-1]+1))
        cost_list = []
        for i in range_SI:
            cost_list.append(cost(k,S,i,adj,holding_cost,processing_time,sigma,Z))
        return min(cost_list),range_SI[cost_list.index(min(cost_list))]   
    elif direction == "i":
        if demand_node(k):
            range_S = list(compress(demand_stage_S, list(map(lambda x:x==k,demand_stage))))
        else:
            range_S = list(range(0,SI + processing_time[k-1]+1))
        cost_list = []
        for i in range_S:
            cost_list.append(cost(k,i,SI,adj,holding_cost,processing_time,sigma,Z))
        return min(cost_list),np.nan
    
def backtrack_func(dp_result,bt,io):
    import numpy as np
    dp_result2 = add_inf(dp_result)
    bt = add_inf(bt)
    results = []
    last_idx = len(io) - 1
    min_index = np.where(dp_result2[:,last_idx]==min(dp_result2[:,last_idx]))[0][0]
    min_val = bt[:,last_idx][min_index]
    results.append(min_val)    
    for i in reversed(range(len(io)-1)):        
        if (io[i] == 'i'):
            min_bt = min(bt[:,i])
            results.append(min_bt)
        else:
            min_val = bt[int(min_val),i]
            results.append(min_val)                      
    return list(reversed(results))
    
def dp_tree():
    """
    Function implementing the Dynamic Programming algorithm for Tree networks
    """
    result = np.zeros((max(M),max(k_new)))
    result[:] = np.nan
    backtrack = np.zeros((max(M),max(k_new)))
    backtrack[:] = np.nan
    theta_type = []
    # iterating over all stages
    for k in range(1,len(k_new)):
        # checking if there is a linked stage with higher index using a custom function
        if higher_linked(k,adj)[0]:
            # setting p to the index number of linked stage (p(k))
            p = higher_linked(k,adj)[1]
        
         # checking if p is downstream of k in supply chain
        if downstream(p,k):                           
            range_S = list(range(M[k-1]+1))
            for i in range_S:
                result[i,k-1] = theta(k,"o",i,i)[0]
                backtrack[i,k-1] = theta(k,"o",i,i)[1]
            theta_type.append('o')
    
    
        else:
            range_SI = list(range(M[k-1]-processing_time[k-1]+1))
            for i in range_SI: 
                result[i,k-1] = theta(k,"i",i,i)[0]
                backtrack[i,k-1] = i
            theta_type.append('i')

    k = max(k_new)
    # checking if node N (last node in set K) is a supply node. The SI is fixed in this case
    if supply_node(k): 
        range_SI = list(compress(supply_stage_SI, list(map(lambda x:x==k,supply_stage))))
    else:
        range_SI = list(range(M[k-1]-processing_time[k-1]+1))
    for i in range_SI:
        result[i,k-1] = theta(k,"i",i,i)[0]
        backtrack[i,k-1] = i
    theta_type.append('i')
    result_final = backtrack_func(result,backtrack,theta_type)
    return result_final,result,backtrack,theta_type

def add_inf(np_mat):
    for i in range(np_mat.shape[0]):
        for j in range(np_mat.shape[1]):
            if np.isnan(np_mat[i,j]):
                np_mat[i,j] = np.inf
    return np_mat




In [3]:
### EXAMPLE 6.5 (Working provided in the theory pdf)
### PROBLEM INPUT
x = [1,2,2]
y = [2,3,4]
adj = create_adj(x,y,4) # custom function to create adjacency matrix
adj,k = relabel(adj)    # relabel() function relabels nodes 


echelon = [1,2,3,3]

holding_cost = [1,2,3,3]
processing_time = [2,1,1,1]
Z = [1,1,1,1]
sigma = [math.sqrt(2),1,math.sqrt(2),1]
M = [3,5,4,5]

# define supply, demand stages
supply_stage = [1]
supply_stage_SI = [1]
demand_stage = [2,4]
demand_stage_S = [0,1]

#print_problem(list(range(1,len(holding_cost)+1)),holding_cost,processing_time,echelon,k)

holding_cost,processing_time,echelon,k_new,k_old =  reorder_labels(holding_cost,processing_time,echelon,by=k)
print_problem(k_old,holding_cost,processing_time,echelon,k_new)

result,dp_result,bt,io = dp_tree()
print(result)
print('\nTheta matrix:\n',dp_result)
print('\nBacktrack matrix:\n',bt)
print('\nStage inbound/outbound matrix:\n',io)


Old Labels/Stage:    [1, 3, 2, 4]
Echelon(Supplier=1): [1, 3, 2, 3]
Local Holding Cost:  [1, 3, 2, 3]
Processing Time:     [2, 1, 1, 1]
New Labels/Stage:    [1, 2, 3, 4]
[1.0, 0.0, 0.0, 0.0]

Theta matrix:
 [[ 2.44948974  3.          8.27791687  8.27791687]
 [ 2.          4.24264069  6.69213043  9.69213043]
 [ 1.41421356  5.19615242  7.19615242 10.93477112]
 [ 0.          6.          7.41421356 11.88828285]
 [        inf  6.70820393  6.70820393 12.69213043]]

Backtrack matrix:
 [[ 1.  0.  0.  0.]
 [ 1.  1.  0.  1.]
 [ 1.  2.  1.  2.]
 [ 1.  3.  2.  3.]
 [inf  4.  3.  4.]]

Stage inbound/outbound matrix:
 ['o', 'i', 'o', 'i']


The theta values of the theta matrix exactly match the theta values obtained in the example solution which confirms that the algorithm works correctly.

In [5]:
### EXAMPLE 6.3 (Working provided in the theory pdf)
### PROBLEM INPUT
x = [3,1]
y = [2,2]
adj = create_adj(x,y,3)
adj,k = relabel(adj)
echelon = [3,2,1]
holding_cost = [7,4,2]
processing_time = [1,0,1]
Z = [1,1,1]
sigma = [1,1,1]
M = [3,2,2]

# define supply, demand stages
supply_stage = [3]
supply_stage_SI = [1]
demand_stage = [1]
demand_stage_S = [1]

#print_problem(list(range(1,len(holding_cost)+1)),holding_cost,processing_time,echelon,k)

holding_cost,processing_time,echelon,k_new,k_old =  reorder_labels(holding_cost,processing_time,echelon,by=k)
print_problem(k_old,holding_cost,processing_time,echelon,k_new)

result,dp_result,bt,io = dp_tree()
print(result)
print('\nTheta matrix:\n',dp_result)
print('\nBacktrack matrix:\n',bt)
print('\nStage inbound/outbound matrix:\n',io)

Old Labels/Stage:    [1, 2, 3]
Echelon(Supplier=1): [3, 2, 1]
Local Holding Cost:  [7, 4, 2]
Processing Time:     [1, 0, 1]
New Labels/Stage:    [1, 2, 3]
[0.0, 0.0, 1.0]

Theta matrix:
 [[0.         0.                inf]
 [7.         4.         2.82842712]
 [9.89949494 5.65685425        inf]]

Backtrack matrix:
 [[ 0.  0. inf]
 [ 1.  1.  1.]
 [ 2.  2. inf]]

Stage inbound/outbound matrix:
 ['i', 'i', 'i']


In [39]:
## EXAMPLE working for create_adj function which creates a graph adjacency matrix from links
x = [1,1,3,4,5,5]
y = [2,3,5,5,6,7]
adj = create_adj(x,y,7)
adj

array([[0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]])